<a href="https://colab.research.google.com/github/karlajagg/Documentacion/blob/main/parameter_learning_tipo_fallas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pgmpy

In [ ]:
import pandas as pd
url= 'https://raw.githubusercontent.com/karlajagg/Documentacion/main/xtrainfallas-tf2.csv'
data=pd.read_csv(url)
print (data)

   AB  ST EHG  AP CHCH  BC CIM PCI  ... CECN SEP  CE BRE ACC COM NOMS Tipo falla
0  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S3  S5   S2          7
1  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S5  S5   S2          2
2  S5  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S2  S5   S2          1
3  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S3  S5   S2          5
4  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S3  S5   S2          6
5  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S3  S5   S2          4
6  S5  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S3  S0   S2          1
7  S3  S5  S5  S5   S5  S5  S4  S5  ...   S5  S5  S5  S4  S5  S5   S5          1

[8 rows x 26 columns]


In [ ]:
#Recortamos elemntos que no presentan relacion causal

data1 = data[['NOMS','ACC','CHC','AB','CHH','COM','Tipo falla']]
print(data1)

  NOMS ACC CHC  AB CHH COM  Tipo falla
0   S2  S3  S5  S3  S5  S5           7
1   S2  S5  S5  S3  S5  S5           2
2   S2  S2  S4  S5  S4  S5           1
3   S2  S3  S5  S3  S5  S5           5
4   S2  S3  S5  S3  S5  S5           6
5   S2  S3  S5  S3  S5  S5           4
6   S2  S3  S5  S5  S5  S0           1
7   S5  S5  S5  S3  S5  S5           1


In [ ]:
from pgmpy.models import BayesianModel

model = BayesianModel([('CHH', 'CHC'),('CHC', 'AB'),('AB', 'Tipo falla'),('CHC', 'ACC'),\
                       ('NOMS', 'ACC'),('NOMS', 'Tipo falla'),('COM', 'AB') ]) 
#El aprendizaje de parámetros es la tarea de estimar los valores de las 
#distribuciones de probabilidad condicional (CPD), 

In [ ]:
model.get_markov_blanket('Tipo falla')

['AB', 'NOMS']

In [ ]:
from pgmpy.estimators import ParameterEstimator
pe = ParameterEstimator(model, data)
#CONTADOR DE ESTADOS
print("\n", pe.state_counts('COM')) 
print("\n", pe.state_counts('AB')) 
print("\n", pe.state_counts('NOMS'))

print("\n", pe.state_counts('ACC')) 
print("\n", pe.state_counts('CHH')) 

print("\n", pe.state_counts('CHC')) 
print("\n", pe.state_counts('Tipo falla')) 


     COM
S0    1
S5    7

 CHC   S4        S5     
COM   S0   S5   S0   S5
AB                     
S3   0.0  0.0  0.0  6.0
S5   0.0  1.0  1.0  0.0

     NOMS
S2     7
S5     1

 CHC    S4        S5     
NOMS   S2   S5   S2   S5
ACC                     
S2    1.0  0.0  0.0  0.0
S3    0.0  0.0  5.0  0.0
S5    0.0  0.0  1.0  1.0

     CHH
S4    1
S5    7

 CHH   S4   S5
CHC          
S4   1.0  0.0
S5   0.0  7.0

 AB           S3        S5     
NOMS         S2   S5   S2   S5
Tipo falla                    
1           0.0  1.0  2.0  0.0
2           1.0  0.0  0.0  0.0
4           1.0  0.0  0.0  0.0
5           1.0  0.0  0.0  0.0
6           1.0  0.0  0.0  0.0
7           1.0  0.0  0.0  0.0


In [ ]:
#Maximum Likelihood Estimation
from pgmpy.estimators import MaximumLikelihoodEstimator
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('COM')) 
print(mle.estimate_cpd('AB')) 
print(mle.estimate_cpd('NOMS')) 
print(mle.estimate_cpd('ACC')) 
print(mle.estimate_cpd('CHH')) 
print(mle.estimate_cpd('CHC')) 
print(mle.estimate_cpd('Tipo falla')) 

+---------+-------+
| COM(S0) | 0.125 |
+---------+-------+
| COM(S5) | 0.875 |
+---------+-------+
+--------+---------+---------+---------+---------+
| CHC    | CHC(S4) | CHC(S4) | CHC(S5) | CHC(S5) |
+--------+---------+---------+---------+---------+
| COM    | COM(S0) | COM(S5) | COM(S0) | COM(S5) |
+--------+---------+---------+---------+---------+
| AB(S3) | 0.5     | 0.0     | 0.0     | 1.0     |
+--------+---------+---------+---------+---------+
| AB(S5) | 0.5     | 1.0     | 1.0     | 0.0     |
+--------+---------+---------+---------+---------+
+----------+-------+
| NOMS(S2) | 0.875 |
+----------+-------+
| NOMS(S5) | 0.125 |
+----------+-------+
+---------+----------+--------------------+---------------------+----------+
| CHC     | CHC(S4)  | CHC(S4)            | CHC(S5)             | CHC(S5)  |
+---------+----------+--------------------+---------------------+----------+
| NOMS    | NOMS(S2) | NOMS(S5)           | NOMS(S2)            | NOMS(S5) |
+---------+----------+------

In [ ]:
# Calibrate all CPDs of `model` using MLE:
model.fit(data, estimator=MaximumLikelihoodEstimator)

In [ ]:
#Bayesian Parameter Estimation
#K2, que simplemente suma 1 al recuento de cada estado

from pgmpy.estimators import BayesianEstimator
est = BayesianEstimator(model, data)

print(est.estimate_cpd('Tipo falla', prior_type='BDeu', equivalent_sample_size=10))

+---------------+---------------------+---------------------+--------------------+---------------------+
| AB            | AB(S3)              | AB(S3)              | AB(S5)             | AB(S5)              |
+---------------+---------------------+---------------------+--------------------+---------------------+
| NOMS          | NOMS(S2)            | NOMS(S5)            | NOMS(S2)           | NOMS(S5)            |
+---------------+---------------------+---------------------+--------------------+---------------------+
| Tipo falla(1) | 0.05555555555555555 | 0.4047619047619048  | 0.537037037037037  | 0.16666666666666669 |
+---------------+---------------------+---------------------+--------------------+---------------------+
| Tipo falla(2) | 0.18888888888888888 | 0.11904761904761907 | 0.0925925925925926 | 0.16666666666666669 |
+---------------+---------------------+---------------------+--------------------+---------------------+
| Tipo falla(4) | 0.18888888888888888 | 0.1190476190476

In [ ]:
# BayesianEstimator, too, can be used via the fit()-method
# Aqui imprimiremos todas las CPDs
model.fit(data, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=10) # default equivalent_sample_size=5
for cpd in model.get_cpds():
    print(cpd)

/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3.6/dist-packages/pgmpy/estimators/BayesianEstimator.py:155: UserWarning: pseudo count specified with bdeu prior. It will be ignored, use dirichlet prior for specifying pseudo_counts
  f"pseudo count specified with {prior_type} prior. It will be ignored, use dirichlet prior for specifying pseudo_counts"
/usr/local/lib/python3

+--------+---------+---------------------+---------------------+---------------------+
| CHC    | CHC(S4) | CHC(S4)             | CHC(S5)             | CHC(S5)             |
+--------+---------+---------------------+---------------------+---------------------+
| COM    | COM(S0) | COM(S5)             | COM(S0)             | COM(S5)             |
+--------+---------+---------------------+---------------------+---------------------+
| AB(S3) | 0.5     | 0.35714285714285715 | 0.35714285714285715 | 0.8529411764705882  |
+--------+---------+---------------------+---------------------+---------------------+
| AB(S5) | 0.5     | 0.6428571428571429  | 0.6428571428571429  | 0.14705882352941177 |
+--------+---------+---------------------+---------------------+---------------------+
+---------+---------------------+---------------------+---------------------+--------------------+
| CHC     | CHC(S4)             | CHC(S4)             | CHC(S5)             | CHC(S5)            |
+---------+--------